In [5]:
# Imports
import sys
import importlib

import pandas as pd
from textblob import TextBlob

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

sys.path.append("..")

import ds_helpers as dh
import helper_funcs as hf
import ml_models as ml

In [6]:
importlib.reload(dh)
importlib.reload(hf)
importlib.reload(ml)

# Fetch all datasets
news_path = "/../../news_data/"
stock_path = "/../../stock_data/"

news_df, stock_meta = dh.fetch_datasets(news_path, stock_path)
stock_dfs = dh.fetch_stock_data(stock_meta, stock_path)

# Prepare datasets
stock_dfs = dh.prep_stock_data(stock_dfs)
news_df = dh.prep_news_data(news_df, stock_dfs)

# Sentiment Analysis
daily_sent = ml.sentiment_analysis(news_df)

Error with stock: AGM$A
Error with stock: CARR.V
Error with stock: UTX.V

Number of Stocks before processing: 	5881
Number of Stocks after processing: 	3511



# XGBoost Modeling

In [1]:
"""
    At this point:
        - stock_dfs: dictionary with ticker as key, DF as value
            - Gather list of tickers being used through stock_dfs.keys()
            - stock_dfs[...]
                - Date, Open, Close, Ticker, r_0d, r_1d, r_7d, r_30d
        - daily_sentiment: DF holding sentiment scores
            - date, category (42 total), avg_sentiment, article_count
    
    The plan:
        - For each time horizon create an XGBoost Model
            - Each model gets all 3511 stock openings and the scores of that day
                - Target is the designated time horizon of that modelSo 
"""
all_rows = []
all_dates = stock_dfs['A']['Date']

cols = ['Date', 'Ticker', 'Open', 'r_0d', 'r_1d', 'r_7d', 'r_30d']
cols.extend(all_cats)

for d in all_dates:
    # Get dictionary of daily sentiments
    sent_df = daily_sentiment[daily_sentiment['date'] == d]
    daily_sent_scores = [sent_df[sent_df['category'] == cat]['avg_sentiment'].item() for cat in all_cats]
    
    # Iterate over tickers
    for ticker, df in stock_dfs.items():
        date, open, close, tick, r0, r1, r7, r30 = df[df['Date'] == d].values[0]
        
        row = [d, tick, open, r0, r1, r7, r30] # Get the date and stock ticker
        row.extend(daily_sent_scores)
        
        all_rows.append(row)

big_df = pd.DataFrame(data=all_rows, columns=cols)

NameError: name 'stock_dfs' is not defined

In [ ]:
feature_cols = ['Open'] + all_cats
X = big_df[feature_cols]

y_0d = big_df['r_0d']
y_1d = big_df['r_1d']
y_7d = big_df['r_7d']
y_30d = big_df['r_30d']

In [ ]:
N_EST = 1000
LAMBDA = 0.05
MAX_DEPTH = 6
EVAL_MET = "mse"

model_0d = xgb.XGBRegressor(
    n_estimators = N_EST,
    learning_rate = LAMBDA,
    max_depth= MAX_DEPTH,
    eval_metric= EVAL_MET
)

model_1d = xgb.XGBRegressor(
    n_estimators = N_EST,
    learning_rate = LAMBDA,
    max_depth= MAX_DEPTH,
    eval_metric= EVAL_MET
)

model_7d = xgb.XGBRegressor(
    n_estimators = N_EST,
    learning_rate = LAMBDA,
    max_depth= MAX_DEPTH,
    eval_metric= EVAL_MET
)

model_30d = xgb.XGBRegressor(
    n_estimators = N_EST,
    learning_rate = LAMBDA,
    max_depth= MAX_DEPTH,
    eval_metric= EVAL_MET
)